# Simple retrieval augmented generation with OpenAI

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True
SAMPLE_COLLECTION_NAME = 'sample_simple_rag'
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'docs'
ID_FIELD = '<var:id_field>' if not APPLY else 'id'
OUTPUT_PREFIX = 'outputs__'
EAGER = False

In [2]:
from superduper import superduper, CFG
import os

db = superduper(initialize_cluster=False)

2025-Feb-24 15:47:12.30| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Feb-24 15:47:12.38| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:50   | Building Data Layer
2025-Feb-24 15:47:12.38| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:67   | Data Layer built
2025-Feb-24 15:47:12.38| INFO     | Duncans-MBP.fritz.box| superduper.base.build:146  | Configuration: 
 +----------------+-----------------------------------+
| Configuration  |               Value               |
+----------------+-----------------------------------+
|  Data Backend  | mongodb://localhost:27017/test_db |
| Artifact Store |   filesystem://./artifact_store   |
+----------------+-----------------------------------+


In [3]:
db.drop(True, True)
db = superduper()

2025-Feb-24 15:47:12.39| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Feb-24 15:47:12.39| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:50   | Building Data Layer
2025-Feb-24 15:47:12.39| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:67   | Data Layer built
2025-Feb-24 15:47:12.40| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:113  | Cluster initialized in 0.01 seconds.
2025-Feb-24 15:47:12.40| INFO     | Duncans-MBP.fritz.box| superduper.base.build:146  | Configuration: 
 +----------------+-----------------------------------+
| Configuration  |               Value               |
+----------------+-----------------------------------+
|  Data Backend  | mongodb://localhost:27017/test_db |
| Artifact Store |   filesystem://./artifact_store   |
+----------------+-----------------------------------+


In [4]:
import json
import requests
import io
from superduper import logging


def getter():
    logging.info('Downloading data...')
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    logging.info('Downloading data... (Done)')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [5]:
if APPLY:
    data = getter()

2025-Feb-24 15:47:12.42| INFO     | Duncans-MBP.fritz.box| __main__:8    | Downloading data...
2025-Feb-24 15:47:13.33| INFO     | Duncans-MBP.fritz.box| __main__:10   | Downloading data... (Done)


<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [6]:
if APPLY:
    from superduper import Document, Table
    table = Table(COLLECTION_NAME, fields={'x': 'str'})
    db.apply(table, force=True)
    ids = db.execute(db[COLLECTION_NAME].insert(data))

2025-Feb-24 15:47:13.38| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-24 15:47:13.38| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-24 15:47:13.38| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-24 15:47:13.42| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, docs) not found in cache, loading from db
2025-Feb-24 15:47:13.42| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'docs')) from metadata...
2025-Feb-24 15:47:13.42| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:369  | Found new Table:docs:e7a8f003e2c945df
2025-Feb-24 15:47:13.42| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:78   | Found these changes and/ or additions that need to be made:
2025-Feb-24 15:47:13.42| INF

Create plugin:

In [7]:
from superduper import Plugin

plugin = Plugin('rag-plugin', path='./rag_plugin.py')

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [8]:
from superduper import Listener
from rag_plugin import Chunker

upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db[COLLECTION_NAME],
    key='x',
    identifier='chunker',
    flatten=True,
    upstream=[plugin],
)

In [9]:
if APPLY and EAGER:
    db.apply(upstream_listener, force=True)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [10]:
import os

from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding',
    model='text-embedding-ada-002',
    datatype='vector[float:1536]',
)

## Create vector-index

In [11]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs],
        model=openai_embedding,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

In [12]:
if APPLY and EAGER:
    db.apply(vector_index, force=True)

<!-- TABS -->
## Build LLM

In [13]:
from superduper_openai import OpenAIChatCompletion


llm_openai = OpenAIChatCompletion(
    identifier='llm-model',
    model='gpt-3.5-turbo',
    datatype='str',
)

## Answer question with LLM

In [14]:
from superduper import model
from rag_plugin import RAGModel


prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "These snippets are samples from our internal data-repositories, and should be used exclusively and as a matter"
    " of priority to answer the question. Please answer in 20 words or less.\n\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

rag = RAGModel(
    'simple_rag',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm_openai,
)

In [15]:
if APPLY and EAGER:
    db.apply(rag, force=True)

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [16]:
from superduper import Streamlit, Plugin
from rag_plugin import demo_func

demo = Streamlit('simple-rag-demo', demo_func=demo_func, upstream=[plugin])

In [17]:
from superduper import Application

app = Application(
    'simple-rag-app',
    components=[
        upstream_listener,
        vector_index,
        rag,
        demo,
    ]
)

In [18]:
if APPLY:
    db.apply(app, force=True)

2025-Feb-24 15:47:13.80| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-24 15:47:13.80| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-24 15:47:13.80| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-24 15:47:13.81| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-24 15:47:13.81| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-24 15:47:13.81| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-24 15:47:13.82| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-24 15:47:13.82

[2025-02-24 15:47:15] httpx INFO HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"
[2025-02-24 15:47:16] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                                     | 0/4 [00:00<?, ?it/s]
[2025-02-24 15:47:18] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                             | 1/4 [00:01<00:04,  1.46s/it]
[2025-02-24 15:47:18] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"████                                                                                                         | 2/4 [00:02<00:02,  1.26s/it]
[2025-02-24 15:47:20] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"██████████████████████████████████████████████████████

2025-Feb-24 15:47:21.09| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, _outputs__embeddinglistener__3034f3f719934df7) not found in cache, loading from db
2025-Feb-24 15:47:21.09| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', '_outputs__embeddinglistener__3034f3f719934df7')) from metadata...
2025-Feb-24 15:47:21.10| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:_outputs__embeddinglistener__3034f3f719934df7:df7a3ef7dc484ed9 to cache
2025-Feb-24 15:47:21.98| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Job) not found in cache, loading from db
2025-Feb-24 15:47:21.98| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Job')) from metadata...
2025-Feb-24 15:47:21.98| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Job:jkl to cache
2025-Feb-24 15:47:22.05| INFO     | Duncans-MBP.f

In [19]:
if APPLY:
    rag = db.load('RAGModel', 'simple_rag')
    print(rag.predict('Tell me about the project'))

2025-Feb-24 15:47:22.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:608  | Getting vector-index
2025-Feb-24 15:47:22.24| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:616  | {}


[2025-02-24 15:47:23] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-02-24 15:47:24] httpx INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Superduper is a system for building, deploying, and managing AI models. It involves architecture, fundamentals, and listening for new data.


You can now load the model elsewhere and make predictions using the following command.

## Create template

In [20]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData

template = Template(
    'simple_rag',
    template=app,
    substitutions={
        COLLECTION_NAME: 'table_name',
        'text-embedding-ada-002': 'embedding_model',
        'gpt-3.5-turbo': 'llm_model',
    },
    template_variables=['table_name', 'id_field', 'embedding_model', 'llm_model'],
    default_tables=[
        Table(
            'sample_simple_rag',
            fields={'x': 'str'},
            data=RemoteData(
                'superduper-docs',
                getter=getter,
            )
        ),
    ],
    types={
        'id_field': {
            'type': 'str',
            'default': '_id',
        },
        'embedding_model': {
            'type': 'str',
            'default': 'text-embedding-ada-002',
            'choices': ['text-embedding-ada-002', 'nomic-embed-text:latest'],
        },
        'llm_model': {
            'type': 'str',
            'default': 'gpt-3.5-turbo',
            'choices': ['gpt-3.5-turbo', 'gpt-4-turbo', 'llama3.1:8b']
        },
        'table_name': {
            'type': 'str',
            'default': SAMPLE_COLLECTION_NAME,
        }
    },
    schema={
        "id_field": "id_field",
        "embedding_model": "embedding_model",
        "llm_model": "llm_model",
        "table_name": "table_name",
    },
    db=db
)

In [21]:
template.export('.')